In [1]:
import pandas as pd
import numpy as np

In [2]:
main_dir = "/projects/robson-lab/research/endometriosis/"
sample_id = "Endometriosis-revision-3"
outdir = f"{main_dir}/analysis/{sample_id}/Supplementary Tables"

---
**edit subcluster naming**

In [3]:
import scanpy as sc
from scanpy.plotting._anndata import _prepare_dataframe
from pathlib import Path

In [4]:
#import & name clusters in datasets
main = sc.read(f"{main_dir}/data/h5ad/{sample_id}/{sample_id}-main-final-cc-20220105.h5ad")
mye = sc.read(f"{main_dir}/data/h5ad/{sample_id}/Myeloid/{sample_id}-mye-final-cc-20220105.h5ad")
lym = sc.read(f"{main_dir}/data/h5ad/{sample_id}/TNK/{sample_id}-lym-final-cc-20220105.h5ad")
epi = sc.read(f"{main_dir}/data/h5ad/{sample_id}/Epithelial/{sample_id}-epi-final-cc-20220119.h5ad")
stro = sc.read(f"{main_dir}/data/h5ad/{sample_id}/Stromal/{sample_id}-stro-final-cc-20220105.h5ad")
endo = sc.read(f"{main_dir}/data/h5ad/{sample_id}/Endothelial/{sample_id}-endo-final-cc-20220105.h5ad")
org = sc.read(f"{main_dir}/data/h5ad/{sample_id}/non_immune/{sample_id}-org-final-cc-20220119.h5ad")

adatas = [main, mye, lym, epi, stro, endo, org]

In [5]:
#subtypes rename
map_names_58 = {
    'glandular' : "glandular proliferative", 
    'lumenal 1': "lumenal 1", 
    'TP63+/KRT5+':"TP63+/KRT5+", 
    'lumenal':"lumenal", 
    'MUC5B+':"MUC5B+", 
    'mid-secretory':"mid-secretory", 
    'ciliated':"ciliated", 
    'glandular early-secretory':"glandular early-secretory", 
    'lumenal 2':"lumenal 2", 
    'mesothelial':"mesothelial",
    'eF3':"eF3", 
    'eF2':"eF2",
    'eF1':"eF1",
    'dS1-myofibroblast':"dS1-myofibroblast", 
    'fib C7':"fib C7", 
    'dS2':"dS2",
    'VSMC':"VSMC", 
    'eF4-CXCL14':"eF4-CXCL14", 
    'Prv-STEAP4':"Prv-STEAP4", 
    'Prv-CCL19':"Prv-CCL19",
    'Prv-MYH11':"Prv-MYH11",
    'fib C7-SFRP2':"fib C7-SFRP2",
    'EC-HEV':"EC-HEV", 
    'EC-tip':"EC-tip", 
    'LEC':"LEC", 
    'EC-artery':"EC-artery", 
    'EC-PCV':"EC-PCV", 
    'EC-aPCV':"EC-aPCV", 
    'EC-capillary':"EC-capillary",
    'monocytes-CD16-':"monocytes-CD16-",      
    'M$\Phi$1-LYVE1':"M1-LYVE1",
    'M$\Phi$4-infiltrated':"M4-infiltrated",
    'M$\Phi$3-APOE':"M3-APOE", 
    'M$\Phi$5-activated': "M5-activated",
    'cDC2':"cDC2", 
    'monocytes-CD16+':"monocytes-CD16+", 
    'pre-cDC2':"pre-cDC2",
    'mast cells':"mast cells",
    'pDC':"pDC",
    'M$\Phi$2-peritoneal':"M2-peritoneal", 
    'DC3':"DC3",
    'cDC1':"cDC1", 
    'mDC':"mDC",
    'granulocytes':"granulocytes",
    'NK1':"NK1",
    'NK2':"NK2", 
    'NK3':"NK3",
    'pNK':"pNK",
    'CTL':"CTL",
    'T$_{EM}$':"TEM", 
    "ILC":"ILC",
    'T$_N$/T$_{CM}$':"TN-TCM",
    'T$_{Reg}$':"TReg",
    'CD4 T$_{RM}$':"CD4-TRM",
    'CD8 T$_{RM}$':"CD8-TRM", 
    "CD8 MAIT":"CD8-MAIT", 
    "plasma":"plasma",
    'B cell':"B cell",
    }

for adata in adatas[1:-1]:
    adata.obs["new_name"] = adata.obs["subtypes"].map(map_names_58)

---
**Supplementary Table 2**: markers for each subpopulation

In [7]:
def report_in_out_group_fractions(
    adata: sc.AnnData,
    key=None,
    groupby=None,
    use_raw=True,
    log=True,
) -> None:
    if key is None:
        key = 'rank_genes_groups'
    if groupby is None:
        groupby = str(adata.uns[key]['params']['groupby'])
    gene_names = pd.DataFrame(adata.uns[key]['names'])
    fraction_in_cluster_matrix = pd.DataFrame(
        np.zeros(gene_names.shape),
        columns=gene_names.columns,
        index=gene_names.index,
    )
    fraction_out_cluster_matrix = pd.DataFrame(
        np.zeros(gene_names.shape),
        columns=gene_names.columns,
        index=gene_names.index,
    )
    for cluster in gene_names.columns:
        var_names = gene_names[cluster].values
        adata.obs['__is_in_cluster__'] = pd.Categorical(adata.obs[groupby] == cluster)
        categories, obs_tidy = _prepare_dataframe(
            adata,
            var_names,
            groupby='__is_in_cluster__',
            use_raw=use_raw,
        )
        mean_obs = obs_tidy.groupby(level=0).mean()
        obs_bool = obs_tidy.astype(bool)
        fraction_obs = obs_bool.groupby(level=0).sum() / obs_bool.groupby(level=0).count()
        fraction_in_cluster_matrix.loc[:, cluster] = fraction_obs.loc[True].values
        fraction_out_cluster_matrix.loc[:, cluster] = fraction_obs.loc[False].values
    adata.obs.drop(columns='__is_in_cluster__')
    return fraction_in_cluster_matrix, fraction_out_cluster_matrix
_helper_outer = lambda df, name: pd.melt(
    df, value_name=name, var_name="cluster"
).set_index("cluster")
_helper = lambda adata, col, name: _helper_outer(
    pd.DataFrame.from_records(adata.uns["rank_genes_groups"][col]), name
)
def compute_marker_genes(adata, groupby="cluster", use_raw=True, method="wilcoxon", suffix="", save=True):
    sc.tl.rank_genes_groups(adata, groupby=groupby, use_raw=use_raw, method="wilcoxon", n_genes=250)
    in_groups, out_groups = report_in_out_group_fractions(adata, groupby=groupby)
    sizes = adata.obs.groupby(groupby).size()
    markers = pd.concat(
        (
            _helper(adata, "names", "gene"),
            _helper(adata, "scores", "score"),
            _helper(adata, "logfoldchanges", "logfc"),
            _helper(adata, "pvals", "pval"),
            _helper(adata, "pvals_adj", "pval_fdr"),
            _helper_outer(in_groups, "in_group_fraction"),
            _helper_outer(out_groups, "out_group_fraction"),
            _helper_outer(out_groups < 0.2, "specific_20%"),
            _helper_outer(out_groups < 0.05, "specific_5%"),
            _helper_outer(out_groups < 0.01, "specific_1%")
        ),
        axis=1
    )
    markers["n_cells"] = sizes.loc[markers.index]
    # remove ribosomal genes
    markers = markers.loc[~markers.gene.str.match("^M?RP[LS]"), :]
    
    if save:
        suffix = f"-{suffix}" if suffix else ""
        #outpath = Path(ANALYSIS_DIR / "markers" / f"{sample_id}{suffix}-nn.csv")
        outpath = f"{ANALYSIS_DIR}/markers/{adata.uns['sampleid']}{suffix}-markers.csv"
        markers.to_csv(outpath, index=True, header=True)
    else:
        return markers

In [8]:
#make dataframe markers for specific_20% & specific 5% (major cell type)

#adatas = [main, mye, lym, epi, stro, endo, org]
celltype = ["myeloid","lymphocytes","epithelial","stromal","endothelial"]

with pd.ExcelWriter(f"{outdir}/markers.xlsx") as writer:
    adata = main.copy()
    adata.raw = adata
    markers = compute_marker_genes(adata, groupby="celltype_main",use_raw=True,save=False)
    df1 = markers[(markers["specific_5%"] == True)]
    df2 = df1.iloc[:,:7]
    df2.to_excel(writer, sheet_name = "major celltype",index = True)
    
    for adata,sheetname in zip(adatas[1:-1], celltype):
        adata_copy = adata.copy()
        adata_copy.raw = adata_copy
        markers = compute_marker_genes(adata_copy, groupby="new_name",use_raw=True,save=False)
        df1 = markers[(markers["specific_20%"] == True)]
        df2 = df1.iloc[:,:7]
        df2.to_excel(writer, sheet_name = f"{sheetname}",index = True)

---
**Supplementary Table 3**: DEG between scRNAseq & bulk

In [22]:
#import data
Eut_deg = pd.read_csv(f"{main_dir}analysis/{sample_id}/bulkRNAseq/DEG-Eutopic-SCvsBS_20220106.csv",
                         index_col  = "genes")

Ect_deg = pd.read_csv(f"{main_dir}analysis/{sample_id}/bulkRNAseq/DEG-Ectopic-SCvsBS_20220106.csv",
                         index_col  = "genes")

Ovr_deg = pd.read_csv(f"{main_dir}analysis/{sample_id}/bulkRNAseq/DEG-Ovary-SCvsBS_20220106.csv",
                         index_col  = "genes")

In [51]:
df = pd.concat([Eut_deg[(Eut_deg.logFC >= 3)][:200],(Eut_deg[(Eut_deg.logFC <= -3)][:200])])

In [52]:
df

,Unnamed: 0,logFC,logCPM,PValue,FDR
genes,,,,,
AC004542.2,249,7.670864,4.411064,1.286577e-77,3.645941e-75
MT-ND4,17472,3.053687,12.778954,5.204836e-68,9.763833e-66
AC004542.1,248,7.742500,3.937699,8.605475e-64,1.340542e-61
MT-ND5,17474,3.008097,12.058954,1.629127e-57,1.946040e-55
MT-ND6,17475,3.082874,10.559769,1.756560e-49,1.448645e-47
...,...,...,...,...,...
BLVRB,7822,-3.139158,6.276720,1.997931e-58,2.475390e-56
TSPO,24805,-3.849433,7.345241,2.949876e-58,3.621143e-56
AC005921.2,497,-8.140601,4.291158,3.427184e-58,4.187767e-56


In [53]:
degs = [Eut_deg, Ect_deg, Ovr_deg]
names = ["Eutopic", "Peritoneal endometriosis", "Ovarian endometrosis"]

with pd.ExcelWriter(f"{outdir}/DEG-scVSbs.xlsx") as writer:
    for data,name in zip(degs,names):
        df = data.drop("Unnamed: 0", axis=1)
        df_slice = pd.concat([df[(df.logFC >= 3)][:200],(df[(df.logFC <= -3)][:200])])
        df_slice.to_excel(writer, sheet_name = f"{name}",index = True)

---
**Source Data**: Extended data 2, QuPAth H&E source data

In [12]:
#read data
data = pd.read_csv(f"{main_dir}analysis/Endometriosis-revision-3/HEstaining/HE_Epi-Proportion.csv")

#transform to long form
df = pd.melt(data, id_vars=["Name","Group"],value_name="Epithelial proportion (%)")

#tidy it up
df["Group"] = df["Group"].astype("category")
df.Group.cat.reorder_categories(["Control","Endometriosis"],inplace=True)

#remove outlier
df1 = df[~df.Name.str.contains("106.32")]
df2 = df1[df1.variable == "Classifier 2"]

In [21]:
df2.to_csv(f"{outdir}/H&E proportion.csv",sep=",")

---
**Supp Table 4**: edgeR DEG between sample type within each subcluster/subtypes/subpopulation

In [6]:
cluster_to_keep = ['dS2','Prv-CCL19','EC-tip','EC-aPCV','EC-PCV','M1','M4','cDC2','Treg',"B cell"]
tab_names= ['dS2','Prv-CCL19','EC-tip','EC-aPCV','EC-PCV','M1-LYVE1','M4-infiltrated','cDC2','TReg',"B cell"]

In [7]:
list(zip(cluster_to_keep, tab_names))

[('dS2', 'dS2'),
 ('Prv-CCL19', 'Prv-CCL19'),
 ('EC-tip', 'EC-tip'),
 ('EC-aPCV', 'EC-aPCV'),
 ('EC-PCV', 'EC-PCV'),
 ('M1', 'M1-LYVE1'),
 ('M4', 'M4-infiltrated'),
 ('cDC2', 'cDC2'),
 ('Treg', 'TReg'),
 ('B cell', 'B cell')]

In [8]:
delete = ["logFC.ecpaVSeco","logFC.ecpVSeco"]
#cols=["logFC.eueVSctr","logFC.ecpVSctr","logFC.ecpaVSctr","logFC.ecoVSctr","logCPM","F","PValue","FDR"]

with pd.ExcelWriter(f"{outdir}/edgeR-output-all-subcluster-top1k.xlsx") as writer:
    for cluster,name in zip(cluster_to_keep, tab_names):
        df = pd.read_csv(f"{main_dir}analysis/{sample_id}/DEG/edgeR-output/{cluster}-DEG.csv", sep=",", index_col=0)
        delete_int = df.columns.intersection(delete)
        if len(delete_int) > 0:
            df.drop(delete_int, axis=1, inplace=True)
        df[:1000].to_excel(writer, sheet_name = f"{name}",index = True, index_label="Gene",
                   #columns=cols
                   )

---
**Supp Table 5**: GSEA, FDR < 10%

In [54]:
subclusters = pd.read_csv(f"{main_dir}analysis/{sample_id}/DEG/edgeR-input/subtypelist.csv", sep=",", index_col=0)["0"].tolist()

In [69]:
original_names = pd.read_csv(f"{main_dir}analysis/{sample_id}/DEG/edgeR-input/subtypelist.csv", sep=",", index_col=0)["0"].tolist()
map_names = {
    'mono (CD16-)':"monocytes-CD16-", 
    'M4':"M4-infiltrated", 
    'M1':"M1-LYVE1", 
    'precDC2':"pre-cDC2", 
    'M3':"M3-APOE", 
    'M5':"M5-activated", 
    'mono (CD16+)':"monocytes-CD16+", 
    'Tem':"TEM", 
    'NK1':"NK1", 
    'CD8Trm':"CD8-TRM", 
    'CD4Trm':"CD4-TRM", 
    'Tncm':"TN-TCM", 
    'B cell':"B cell", 
    'NK2':"NK2", 
    'NK3':"NK3", 
    'Treg':"TReg", 
    'CTL':"CTL", 
    'glandular':"glandular proliferative",  
    }
    
new_name = [map_names.get(x, x) for x in original_names]
#new_name

In [71]:
#df = pd.DataFrame()
#pairs = ["eueVSctr","ecpVSctr","ecpaVSctr","ecoVSctr"]
tpairs = ["EuE vs Ctrl","EcP vs Ctrl","EcPA vs Ctrl","EcO vs Ctrl"]

with pd.ExcelWriter(f"{outdir}/gsea-output.xlsx") as writer:
    for cluster,name in zip(subclusters,new_name):
        get_listofpairs = pd.read_csv(f"{main_dir}analysis/{sample_id}/DEG/edgeR-output/{cluster}-DEG.csv", sep=",", index_col=0)
        pairs = get_listofpairs.loc[:,get_listofpairs.columns.str.contains("logFC")].columns.str.lstrip("logFC.").tolist()
        removes = {"ecpVSeco","ecpaVSeco"}
        pairs = [x for x in pairs if x not in removes]
        df = pd.DataFrame()
        
        for i,j in zip(pairs, tpairs):
            data = pd.read_csv(f"{main_dir}analysis/{sample_id}/DEG/GSEA-output/GOBP-filtered/gsea_{cluster}-logFC.{i}.csv")
            data.insert(0,"tissue_pair",f"{j}")
            df = df.append(data)
            df.to_excel(writer, sheet_name = f"{name}")
            print(len(df))

Exception ignored in: <function ZipFile.__del__ at 0x2aaad969bf70>
Traceback (most recent call last):
  File "/home/tany/.conda/envs/endometriosis/lib/python3.8/zipfile.py", line 1821, in __del__
    self.close()
  File "/home/tany/.conda/envs/endometriosis/lib/python3.8/zipfile.py", line 1838, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file


53
75
91
122
70
144
223
286
37
65
130
175
35
61
156
230
80
206
269
354
15
16
47
55
10
16
19
19
39
63
137
252
19
19
34
59
47
97
177
205
2
2
20
38
3
3
22
40
1
1
1
20
17
37
58
68
1
5
25
51
20
38
56
74
24
42
44
68
18
18
41
41
0
0
0
2
143
362
410
152
174
217
250
57
91
139
189
129
188
248
58
91
119
205
12
41
58
79
60
118
122
124
36
43
43
39
47
81
82
148
213
310
460
266
359
439
519
69
91
108
188
59
103
151
198
8
31
38
78
6
25
36
132
59
140
166
280
29
118
224
296
42
112
197
262
23
47
75
111
36
41
55
72
22
32
32
42
0
0
1
1
39
42
49
58
9
26
37
62
19
20
21
21
3
3
5
6
2
2
9
11


---
Supp Table : CellPhone DB unique interactions (<150 counts)

In [74]:
cellphone = pd.read_csv(f"{main_dir}/analysis/{sample_id}/cellphone/all-celltypes-2/annotated_results_20220125.csv",
                       index_col=0)

/home/tany/.conda/envs/endometriosis/lib/python3.8/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [82]:
unique_df =  cellphone[(cellphone.pval < 0.01) &\
          (cellphone.frac_samples_observed >= 0.5) &\
          (cellphone.self_interaction == False) &\
          (cellphone.n_celltype_pairs <=150)]

unique_df.to_csv(f"{main_dir}/analysis/{sample_id}/cellphone/filtered_unique_interactions.csv", sep=",")

In [156]:
unique_df["celltype_a_2"] = unique_df.celltype_a.map(map_names_58)
unique_df["celltype_b_2"] = unique_df.celltype_b.map(map_names_58)
unique_df["celltype_pair"] = unique_df.celltype_a_2 + " -> " + unique_df.celltype_b_2

grp = unique_df.groupby(["sample_type", "interacting_pair"])
cellphone_table = grp.aggregate(dict(
    frac_samples_observed=lambda p: p.iloc[0],
    n_celltype_pairs=lambda p: p.iloc[0],
    celltype_pair=' / '.join,
)).reset_index()

<ipython-input-156-4814333324a5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_df["celltype_a_2"] = unique_df.celltype_a.map(map_names_58)
<ipython-input-156-4814333324a5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_df["celltype_b_2"] = unique_df.celltype_b.map(map_names_58)
<ipython-input-156-4814333324a5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

,sample_type,interacting_pair,frac_samples_observed,n_celltype_pairs,celltype_pair
0,EcO,ACKR1_CCL17,0.750000,33,EC-HEV -> cDC2 / EC-PCV -> cDC2 / EC-aPCV -> c...
1,EcO,ACKR2_CCL14,0.500000,26,LEC -> EC-HEV / LEC -> EC-PCV / LEC -> EC-aPCV...
2,EcO,ACKR2_CCL2,0.500000,40,LEC -> EC-tip / LEC -> M4-infiltrated / LEC ->...
3,EcO,ACKR2_CCL28,0.500000,26,LEC -> NK3 / LEC -> NK3
4,EcO,ACKR2_CCL3,0.500000,42,LEC -> M1-LYVE1 / LEC -> NK3 / LEC -> TEM / LE...
...,...,...,...,...,...
797,control,WNT5A_EPHA7,0.666667,106,dS1-myofibroblast -> eF2 / dS1-myofibroblast -...
798,control,WNT7A_FZD5,0.666667,79,glandular proliferative -> M5-activated / glan...
799,control,WNT7B_FZD1,0.666667,46,MUC5B+ -> M3-APOE / MUC5B+ -> Prv-CCL19 / MUC5...
800,control,WNT7B_FZD10,0.666667,30,MUC5B+ -> Prv-CCL19 / MUC5B+ -> Prv-MYH11 / MU...


In [158]:
cellphone_table = grp.aggregate(dict(
    frac_samples_observed=lambda p: p.iloc[0],
    n_celltype_pairs=lambda p: p.iloc[0],
    celltype_pair=' / '.join,
)).reset_index()

cellphone_table.to_csv(f"{main_dir}/analysis/{sample_id}/Supplementary Tables/unique_interactions.csv", sep=",")